In [1]:
import os
os.environ["KERAS_BACKEND"] = "torch"

In [ ]:
import torch
import torchvision
import keras
import warnings

from torchvision import datasets
from torchvision.transforms import v2 as transforms
from typing import *

warnings.filterwarnings("ignore")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {device}")

Using device: cuda



## About the dataset

The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.

The dataset is divided into five training batches and one test batch, each with 10000 images. The test batch contains exactly 1000 randomly-selected images from each class. The training batches contain the remaining images in random order, but some training batches may contain more images from one class than another. Between them, the training batches contain exactly 5000 images from each class.

The classes are:

| Label |	Description |
|-------|-------------|
| 0     |	airplane    |
| 1     |	automobile  |
| 2     |	bird        |
| 3     |	cat         |
| 4     |	deer        |
| 5     |	dog         |
| 6     |	frog        |
| 7     |	horse       |
| 8     |	ship        |
| 9     |	truck       |

This dataset is just like the CIFAR-10, except it has 100 classes containing 600 images each. There are 500 training images and 100 testing images per class. The 100 classes in the CIFAR-100 are grouped into 20 superclasses. Each image comes with a "fine" label (the class to which it belongs) and a "coarse" label (the superclass to which it belongs).
Here is the list of classes in the CIFAR-100:

| Superclass |	Classes |
|------------|----------|
|aquatic mammals|	beaver, dolphin, otter, seal, whale|
|fish|	aquarium fish, flatfish, ray, shark, trout|
|flowers|	orchids, poppies, roses, sunflowers, tulips|
|food containers|	bottles, bowls, cans, cups, plates|
|fruit and vegetables|	apples, mushrooms, oranges, pears, sweet peppers|
|household electrical devices|	clock, computer keyboard, lamp, telephone, television|
|household furniture|	bed, chair, couch, table, wardrobe|
|insects|	bee, beetle, butterfly, caterpillar, cockroach|
|large carnivores|	bear, leopard, lion, tiger, wolf|
|large man-made outdoor things|	bridge, castle, house, road, skyscraper|
|large natural outdoor scenes|	cloud, forest, mountain, plain, sea|
|large omnivores and herbivores|	camel, cattle, chimpanzee, elephant, kangaroo|
|medium-sized mammals|	fox, porcupine, possum, raccoon, skunk|
|non-insect invertebrates|	crab, lobster, snail, spider, worm|
|people|	baby, boy, girl, man, woman|
|reptiles|	crocodile, dinosaur, lizard, snake, turtle|
|small mammals|	hamster, mouse, rabbit, shrew, squirrel|
|trees|	maple, oak, palm, pine, willow|
|vehicles 1|	bicycle, bus, motorcycle, pickup truck, train|
|vehicles 2|	lawn-mower, rocket, streetcar, tank, tractor|


Source: https://www.cs.toronto.edu/~kriz/cifar.html

## CIFAR-10

### Pytorch CNN

In [3]:
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(45),
    transforms.RandomResizedCrop(32),
    transforms.Normalize((0.5, 0.5, 0.5), (1.0, 1.0, 1.0)),
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (1.0, 1.0, 1.0)),
])


target_transform = transforms.Lambda(lambda x: torch.nn.functional.one_hot(torch.tensor(x).long(), num_classes=10).float())

train = datasets.CIFAR10(
    "cifar10",
    train=True,
    transform=train_transform,
    target_transform=target_transform,
    download=True
)

test  = datasets.CIFAR10("cifar10",
    train=False,
    transform=test_transform,
    target_transform=target_transform,
    download=True
)

trainloader = torch.utils.data.DataLoader(train, batch_size=128,  shuffle=True)
testloader  = torch.utils.data.DataLoader(test,  batch_size=128, shuffle=False)

print(f"Train size: {len(train)}")
print(f"Test size: {len(test)}")

print(f"Image shape: {train[0][0].shape}")

Files already downloaded and verified
Files already downloaded and verified
Train size: 50000
Test size: 10000
Image shape: torch.Size([3, 32, 32])


In [10]:
model = torch.nn.Sequential(
    torch.nn.Conv2d(3, 32, 3, 1, 1),
    torch.nn.ReLU(),
    torch.nn.BatchNorm2d(32),
    torch.nn.MaxPool2d(2, 2),
    torch.nn.Conv2d(32, 64, 3, 1, 1),
    torch.nn.ReLU(),
    torch.nn.BatchNorm2d(64),
    torch.nn.MaxPool2d(2, 2),
    torch.nn.Flatten(),
    torch.nn.Linear(4096, 1024),
    torch.nn.ReLU(),
    torch.nn.Linear(1024, 10),
)

print(model)
print("-" * 20, " Training ", "-" * 20)

model.to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
criterion = torch.nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.8)

epochs = 10
for epoch in range(epochs):
    model.train()
    train_loss = 0
    train_acc  = 0
    for batch, (images, labels) in enumerate(trainloader):
        images, labels = images.to(device), labels.to(device)

        y_pred = model(images)
        loss = criterion(y_pred, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_acc  += (y_pred.argmax(dim=1) == labels.argmax(dim=1)).sum().item()

    train_loss /= batch + 1
    train_acc  /= trainloader.batch_size * (batch + 1)

    model.eval()
    
    with torch.no_grad():
        test_loss = 0
        accuracy  = 0
        for batch, (images, labels) in enumerate(testloader):
            images, labels = images.to(device), labels.to(device)

            y_pred = model(images)
            loss = criterion(y_pred, labels)

            test_loss += loss.item()
            accuracy  += (y_pred.argmax(dim=1) == labels.argmax(dim=1)).sum().item()

        test_loss /= batch + 1
        accuracy  /= testloader.batch_size * (batch + 1)
    
    print(f"Epoch: {epoch + 1}")
    print(f"Train loss: {train_loss:.4f}, accuracy: {train_acc:.4f}")
    print(f"Test loss: {test_loss:.4f}, accuracy: {accuracy:.4f}")
    print("-" * 50)
    
    scheduler.step()

Sequential(
  (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU()
  (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): ReLU()
  (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (8): Flatten(start_dim=1, end_dim=-1)
  (9): Linear(in_features=4096, out_features=1024, bias=True)
  (10): ReLU()
  (11): Linear(in_features=1024, out_features=10, bias=True)
)
--------------------  Training  --------------------
Epoch: 1
Train loss: 1.8530, accuracy: 0.3183
Test loss: 1.5967, accuracy: 0.4278
--------------------------------------------------
Epoch: 2
Train loss: 1.6923, accuracy: 0.3794
Test loss: 1.5454, accuracy: 0.4623
-----------------------------

### Keras CNN

In [3]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

x_train = x_train / 255
x_test  = x_test / 255

augment = keras.Sequential([
    keras.layers.RandomRotation(0.2),
    keras.layers.RandomZoom(0.2),
    keras.layers.RandomFlip(),
])

x_train = augment(x_train)

print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape}")
print(f"y_test shape: {y_test.shape}")  

x_train shape: torch.Size([50000, 32, 32, 3])
y_train shape: (50000, 1)
x_test shape: (10000, 32, 32, 3)
y_test shape: (10000, 1)


In [4]:

model = keras.models.Sequential()

model.add(keras.layers.Input(shape=(32, 32, 3)))

model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D((2, 2)))

model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D((2, 2)))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(1024, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(10, activation='softmax'))

print("-" * 20, " Training ", "-" * 20)

model.compile(
    optimizer=keras.optimizers.SGD(learning_rate=0.01, momentum=0.9),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=[
        keras.metrics.SparseCategoricalAccuracy()
    ]
)

model.fit(
    x=x_train, 
    y=y_train,
    epochs=10,
    validation_data=(x_test, y_test),
    batch_size=32,
    shuffle=True,
)

--------------------  Training  --------------------
Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 26s 16ms/step - loss: 2.3144 - sparse_categorical_accuracy: 0.2649 - val_loss: 1.6728 - val_sparse_categorical_accuracy: 0.3954
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 24s 15ms/step - loss: 1.7051 - sparse_categorical_accuracy: 0.3862 - val_loss: 1.4701 - val_sparse_categorical_accuracy: 0.4670
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 28s 18ms/step - loss: 1.5920 - sparse_categorical_accuracy: 0.4320 - val_loss: 1.5970 - val_sparse_categorical_accuracy: 0.4378
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 25s 16ms/step - loss: 1.4921 - sparse_categorical_accuracy: 0.4666 - val_loss: 1.4970 - val_sparse_categorical_accuracy: 0.4639
Epoch 5/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 27s 17ms/step - loss: 1.4281 - sparse_categorical_accuracy: 0.4927 - val_loss: 1.3773 - val_sparse_categorical_accuracy: 0.5013
Epoch 6/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 32s 21ms/step - loss: 1.3535 - sparse_categorical_accur

### ResNet50 Transfet Learning

In [14]:
train_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(45),
    transforms.Normalize((0.5, 0.5, 0.5), (1.0, 1.0, 1.0)),
])

test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (1.0, 1.0, 1.0)),
])


target_transform = transforms.Lambda(lambda x: torch.nn.functional.one_hot(torch.tensor(x).long(), num_classes=10).float())

train = datasets.CIFAR10(
    "cifar10",
    train=True,
    transform=train_transform,
    target_transform=target_transform,
    download=True
)

test  = datasets.CIFAR10("cifar10",
    train=False,
    transform=test_transform,
    target_transform=target_transform,
    download=True
)

trainloader = torch.utils.data.DataLoader(train, batch_size=128,  shuffle=True)
testloader  = torch.utils.data.DataLoader(test,  batch_size=128, shuffle=False)

print(f"Train size: {len(train)}")
print(f"Test size: {len(test)}")

print(f"Image shape: {train[0][0].shape}")

Files already downloaded and verified
Files already downloaded and verified
Train size: 50000
Test size: 10000
Image shape: torch.Size([3, 224, 224])


In [17]:
resnet = torchvision.models.resnet18(pretrained=True)

for param in resnet.parameters():
    param.requires_grad = False

print("-" * 20, " ResNet50 ", "-" * 20)
print("-" * 20, " Training ", "-" * 20)

resnet.fc = torch.nn.Linear(512, 10)

resnet.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet.fc.parameters(), lr=0.0001, betas=(0.9, 0.9))

epochs = 10

for epoch in range(epochs):
    resnet.train()
    train_loss = 0
    train_acc  = 0
    for batch, (images, labels) in enumerate(trainloader):
        images, labels = images.to(device), labels.to(device)

        y_pred = resnet(images)
        loss = criterion(y_pred, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_acc  += (y_pred.argmax(dim=1) == labels.argmax(dim=1)).sum().item()

    train_loss /= batch + 1
    train_acc  /= trainloader.batch_size * (batch + 1)

    resnet.eval()
    with torch.no_grad():
        test_loss = 0
        accuracy  = 0
        for batch, (images, labels) in enumerate(testloader):
            images, labels = images.to(device), labels.to(device)

            y_pred = resnet(images)
            loss = criterion(y_pred, labels)

            test_loss += loss.item()
            accuracy  += (y_pred.argmax(dim=1) == labels.argmax(dim=1)).sum().item()

        test_loss /= batch + 1
        accuracy  /= testloader.batch_size * (batch + 1)
    
    print(f"Epoch: {epoch + 1}")
    print(f"Train loss: {train_loss:.4f}, accuracy: {train_acc:.4f}")
    print(f"Test loss: {test_loss:.4f}, accuracy: {accuracy:.4f}")
    print("-" * 50)


--------------------  ResNet50  --------------------
--------------------  Training  --------------------
Epoch: 1
Train loss: 2.0252, accuracy: 0.2930
Test loss: 1.7218, accuracy: 0.4437
--------------------------------------------------
Epoch: 2
Train loss: 1.5836, accuracy: 0.4903
Test loss: 1.4162, accuracy: 0.5484
--------------------------------------------------
Epoch: 3
Train loss: 1.4025, accuracy: 0.5427
Test loss: 1.2732, accuracy: 0.5777
--------------------------------------------------
Epoch: 4
Train loss: 1.3088, accuracy: 0.5668
Test loss: 1.1808, accuracy: 0.6057
--------------------------------------------------
Epoch: 5
Train loss: 1.2502, accuracy: 0.5803
Test loss: 1.1209, accuracy: 0.6211
--------------------------------------------------
Epoch: 6
Train loss: 1.2150, accuracy: 0.5890
Test loss: 1.0621, accuracy: 0.6480
--------------------------------------------------
Epoch: 7
Train loss: 1.1835, accuracy: 0.5991
Test loss: 1.0571, accuracy: 0.6439
--------------

## CIFAR-100

### Pytorch CNN

In [18]:

train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(45),
    transforms.RandomResizedCrop(32),
    transforms.Normalize((0.5, 0.5, 0.5), (1.0, 1.0, 1.0)),
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (1.0, 1.0, 1.0)),
])


target_transform = transforms.Lambda(lambda x: torch.nn.functional.one_hot(torch.tensor(x).long(), num_classes=100).float())

train = datasets.CIFAR100(
    "cifar100",
    train=True,
    transform=train_transform,
    target_transform=target_transform,
    download=True
)

test  = datasets.CIFAR100(
    "cifar100",
    train=False,
    transform=test_transform,
    target_transform=target_transform,
    download=True
)

trainloader = torch.utils.data.DataLoader(train, batch_size=128,  shuffle=True)
testloader  = torch.utils.data.DataLoader(test,  batch_size=128, shuffle=False)

print(f"Train size: {len(train)}")
print(f"Test size: {len(test)}")

print(f"Image shape: {train[0][0].shape}")

100%|██████████| 169M/169M [00:49<00:00, 3.42MB/s] 


Extracting cifar100/cifar-100-python.tar.gz to cifar100
Files already downloaded and verified
Train size: 50000
Test size: 10000
Image shape: torch.Size([3, 32, 32])


In [25]:
model = torch.nn.Sequential(
    torch.nn.Conv2d(3, 32, 3, 1, 1),
    torch.nn.ReLU(),
    torch.nn.BatchNorm2d(32),
    torch.nn.MaxPool2d(2, 2),
    torch.nn.Conv2d(32, 64, 3, 1, 1),
    torch.nn.ReLU(),
    torch.nn.BatchNorm2d(64),
    torch.nn.MaxPool2d(2, 2),
    torch.nn.Flatten(),
    torch.nn.Linear(4096, 1024),
    torch.nn.ReLU(),
    torch.nn.Linear(1024, 100),
)

print(model)
print("-" * 20, " Training ", "-" * 20)

model.to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=0.05, momentum=0.9)
criterion = torch.nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.8)

epochs = 5
for epoch in range(epochs):
    model.train()
    train_loss = 0
    train_acc  = 0
    for batch, (images, labels) in enumerate(trainloader):
        images, labels = images.to(device), labels.to(device)

        y_pred = model(images)
        loss = criterion(y_pred, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_acc  += (y_pred.argmax(dim=1) == labels.argmax(dim=1)).sum().item()

    train_loss /= batch + 1
    train_acc  /= trainloader.batch_size * (batch + 1)

    model.eval()
    
    with torch.no_grad():
        test_loss = 0
        accuracy  = 0
        for batch, (images, labels) in enumerate(testloader):
            images, labels = images.to(device), labels.to(device)

            y_pred = model(images)
            loss = criterion(y_pred, labels)

            test_loss += loss.item()
            accuracy  += (y_pred.argmax(dim=1) == labels.argmax(dim=1)).sum().item()

        test_loss /= batch + 1
        accuracy  /= testloader.batch_size * (batch + 1)
    
    print(f"Epoch: {epoch + 1}")
    print(f"Train loss: {train_loss:.4f}, accuracy: {train_acc:.4f}")
    print(f"Test loss: {test_loss:.4f}, accuracy: {accuracy:.4f}")
    print("-" * 50)
    
    scheduler.step()

Sequential(
  (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU()
  (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): ReLU()
  (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (8): Flatten(start_dim=1, end_dim=-1)
  (9): Linear(in_features=4096, out_features=1024, bias=True)
  (10): ReLU()
  (11): Linear(in_features=1024, out_features=100, bias=True)
)
--------------------  Training  --------------------
Epoch: 1
Train loss: 4.0312, accuracy: 0.0872
Test loss: 3.6431, accuracy: 0.1480
--------------------------------------------------
Epoch: 2
Train loss: 3.6472, accuracy: 0.1444
Test loss: 3.4635, accuracy: 0.1847
----------------------------

### Keras CNN

In [26]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data()

x_train = x_train / 255
x_test  = x_test / 255

augment = keras.Sequential([
    keras.layers.RandomRotation(0.2),
    keras.layers.RandomZoom(0.2),
    keras.layers.RandomFlip(),
])

x_train = augment(x_train)

print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape}")
print(f"y_test shape: {y_test.shape}")  

169001437/169001437 ━━━━━━━━━━━━━━━━━━━━ 49s 0us/step
x_train shape: torch.Size([50000, 32, 32, 3])
y_train shape: (50000, 1)
x_test shape: (10000, 32, 32, 3)
y_test shape: (10000, 1)


In [30]:

model = keras.models.Sequential()

model.add(keras.layers.Input(shape=(32, 32, 3)))

model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D((2, 2)))

model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D((2, 2)))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(1024, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(100, activation='softmax'))

print("-" * 20, " Training ", "-" * 20)

model.compile(
    optimizer=keras.optimizers.SGD(learning_rate=0.01, momentum=0.9),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=[
        keras.metrics.SparseCategoricalAccuracy()
    ]
)

model.fit(
    x=x_train, 
    y=y_train,
    epochs=5,
    validation_data=(x_test, y_test),
    batch_size=32,
    shuffle=True,
)

--------------------  Training  --------------------
Epoch 1/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 24s 15ms/step - loss: 4.4709 - sparse_categorical_accuracy: 0.0628 - val_loss: 3.6481 - val_sparse_categorical_accuracy: 0.1408
Epoch 2/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 23s 15ms/step - loss: 3.6720 - sparse_categorical_accuracy: 0.1335 - val_loss: 3.4498 - val_sparse_categorical_accuracy: 0.1737
Epoch 3/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 23s 15ms/step - loss: 3.4684 - sparse_categorical_accuracy: 0.1730 - val_loss: 3.4240 - val_sparse_categorical_accuracy: 0.1861
Epoch 4/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 22s 14ms/step - loss: 3.3006 - sparse_categorical_accuracy: 0.2005 - val_loss: 3.2106 - val_sparse_categorical_accuracy: 0.2227
Epoch 5/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 23s 14ms/step - loss: 3.1793 - sparse_categorical_accuracy: 0.2210 - val_loss: 3.1118 - val_sparse_categorical_accuracy: 0.2357


### ResNet50 Transfet Learning

In [32]:
train_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(45),
    transforms.Normalize((0.5, 0.5, 0.5), (1.0, 1.0, 1.0)),
])

test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (1.0, 1.0, 1.0)),
])


target_transform = transforms.Lambda(lambda x: torch.nn.functional.one_hot(torch.tensor(x).long(), num_classes=100).float())

train = datasets.CIFAR100(
    "cifar100",
    train=True,
    transform=train_transform,
    target_transform=target_transform,
    download=True
)

test  = datasets.CIFAR100(
    "cifar100",
    train=False,
    transform=test_transform,
    target_transform=target_transform,
    download=True
)

trainloader = torch.utils.data.DataLoader(train, batch_size=128,  shuffle=True)
testloader  = torch.utils.data.DataLoader(test,  batch_size=128, shuffle=False)

print(f"Train size: {len(train)}")
print(f"Test size: {len(test)}")

print(f"Image shape: {train[0][0].shape}")

Files already downloaded and verified
Files already downloaded and verified
Train size: 50000
Test size: 10000
Image shape: torch.Size([3, 224, 224])


In [34]:
resnet = torchvision.models.resnet18(pretrained=True)

for param in resnet.parameters():
    param.requires_grad = False

print("-" * 20, " ResNet50 ", "-" * 20)
print("-" * 20, " Training ", "-" * 20)

resnet.fc = torch.nn.Linear(512, 100)

resnet.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet.fc.parameters(), lr=0.001, betas=(0.9, 0.9))

epochs = 5

for epoch in range(epochs):
    resnet.train()
    train_loss = 0
    train_acc  = 0
    for batch, (images, labels) in enumerate(trainloader):
        images, labels = images.to(device), labels.to(device)

        y_pred = resnet(images)
        loss = criterion(y_pred, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_acc  += (y_pred.argmax(dim=1) == labels.argmax(dim=1)).sum().item()

    train_loss /= batch + 1
    train_acc  /= trainloader.batch_size * (batch + 1)

    resnet.eval()
    with torch.no_grad():
        test_loss = 0
        accuracy  = 0
        for batch, (images, labels) in enumerate(testloader):
            images, labels = images.to(device), labels.to(device)

            y_pred = resnet(images)
            loss = criterion(y_pred, labels)

            test_loss += loss.item()
            accuracy  += (y_pred.argmax(dim=1) == labels.argmax(dim=1)).sum().item()

        test_loss /= batch + 1
        accuracy  /= testloader.batch_size * (batch + 1)
    
    print(f"Epoch: {epoch + 1}")
    print(f"Train loss: {train_loss:.4f}, accuracy: {train_acc:.4f}")
    print(f"Test loss: {test_loss:.4f}, accuracy: {accuracy:.4f}")
    print("-" * 50)


--------------------  ResNet50  --------------------
--------------------  Training  --------------------
Epoch: 1
Train loss: 3.2599, accuracy: 0.2500
Test loss: 2.5970, accuracy: 0.3580
--------------------------------------------------
Epoch: 2
Train loss: 2.5003, accuracy: 0.3703
Test loss: 2.2858, accuracy: 0.4137
--------------------------------------------------
Epoch: 3
Train loss: 2.3272, accuracy: 0.3987
Test loss: 2.1685, accuracy: 0.4401
--------------------------------------------------
Epoch: 4
Train loss: 2.2492, accuracy: 0.4149
Test loss: 2.1331, accuracy: 0.4418
--------------------------------------------------
Epoch: 5
Train loss: 2.2015, accuracy: 0.4221
Test loss: 2.0690, accuracy: 0.4564
--------------------------------------------------


---